In [ ]:
import numpy as np
import scipy.io as io
import matplotlib.pyplot as plt
from scipy.stats import norm
import netCDF4 as n4
import xarray
from scipy.interpolate import griddata
from sklearn.linear_model import LinearRegression
import math

In [ ]:
def Div(x, var):
    Re = 6.37e6
    return 1/(2*np.pi*np.square(Re))*np.gradient(var,x)

In [ ]:
def normalize(x,y):
    xmin = np.min(x)
    xmax = np.max(x)
    ymin = np.min(y)
    ymax = np.max(y)
    '''x_norm = (x-xmin)/(xmax-xmin)
    xcoef = np.mean(x_norm)/np.mean(x)
    y_norm = (y-ymin)/(ymax-ymin)
    ycoef = np.mean(y_norm)/np.mean(y)'''
    
    x_norm = x/np.mean(x)*np.mean(y)
    xcoef = np.mean(x_norm)/np.mean(x)
    y_norm = y
    ycoef = np.mean(y_norm)/np.mean(y)
    norm_ratio = np.mean(y)/np.mean(x)
    return x_norm, y_norm, norm_ratio

def TLS_nointercept(x,y):
    model = LinearRegression(fit_intercept=False, copy_X=False)
    res = model.fit(x.reshape(-1,1),y.reshape(-1,1))
    return res.coef_, res.intercept_

def TLS_intercept(x,y):
    model = LinearRegression(fit_intercept=True, copy_X=False)
    res = model.fit(x.reshape(-1,1),y.reshape(-1,1))
    return res.coef_[0][0], res.intercept_[0] 

In [ ]:
def Regression(var1, var2, x):
    x_lat_ind = np.arange(0,101,1)
    x1_ind = x_lat_ind[0:10]
    x2_ind = x_lat_ind[10:20]
    x3_ind = x_lat_ind[20:30]
    x4_ind = x_lat_ind[30:40]
    x5_ind = x_lat_ind[40:50]
    x6_ind = x_lat_ind[51:61]
    x7_ind = x_lat_ind[61:71]
    x8_ind = x_lat_ind[71:81]
    x9_ind = x_lat_ind[81:91]
    x10_ind = x_lat_ind[91:]
    ind_list = [x1_ind,x2_ind,x3_ind,x4_ind,x5_ind,x6_ind,x7_ind,x8_ind,x9_ind,x10_ind]
    
    a_bins = np.zeros(10)
    b_bins = np.zeros(10)
    
    for i in range(10):
        Y = var2[ind_list[i]]
        X = var1[ind_list[i]]
        nan_mask = np.isnan(X)
        X = X[~nan_mask]
        Y = Y[~nan_mask]
        xcoef, ycoef, norm_ratio = normalize(X,Y)
        a, b = TLS_nointercept(xcoef,ycoef)
        a = a * norm_ratio
        b = b * norm_ratio
        a_bins[i] = a
        b_bins[i] = b
    
    x_center = (x[np.array([4,14,24,34,44,55,65,75,85,95])]+x[np.array([5,15,25,35,45,56,66,76,86,96])])/2
    return a_bins, b_bins, x_center

In [ ]:
def operate(R_frc, T_ctrl, B, G_frc):
    #time step in fraction of year
    delt=1.33e-6
    NMAX=29e6;

    #set up x array (x = sine of latitude).
    jmx=101; #101
    delx = 2.0/jmx;
    x = np.arange(-1.0+delx/2,1.0,delx)
    phi = np.arcsin(x)*180/np.pi;

    # I think this C = rho * c * h_ml /(pi*1e7).
    # this is consistent with a ~1m layer of dirt
    # Note - heat capacity over LAND for fast convergence
    Cl = 0.2; # units: J /(m2 K)

    # Moisture parameters
    relhum = 0.8;   # relative humidity
    eps = 0.622;    # moisture coonstant
    psfc = 9.8e4;   # (Pa)
    e0 = 611.2;     # vap. press (Pa)
    a = 17.67; b = 243.5;   # sat vap constants !!T must be in temperature
    L = 2.45e6;         # latent heat of vaporization (J kg-1)
    cp = 1004;          # (J kg-1 K-1)

    # magnitude of diffusivity
    Dmag = 0.3; # D = 0.2598 W/(m2 K) is the value used by TF10 
    D=Dmag*np.ones(jmx+1); # diffusivity for MSE
    
    q_ctrl = eps*relhum/psfc*e0*np.exp(a*(T_ctrl)/(b+(T_ctrl)))
    theta_e_ctrl = 1/cp*(cp*((T_ctrl)+273.15) + L*q_ctrl)
    
    #Create matrix to take a divergence of something it acts on   
    #set up lambda array.
    lam=(1-np.square(np.arange(-1.0,1.0+delx,delx)))/np.square(delx);
    lam=np.multiply(D, lam); 

    M = np.zeros((jmx,jmx));

    M[0,0] =  - lam[1];
    M[0,1] = lam[1];

    M[jmx-1,jmx-2] = lam[jmx-1];
    M[jmx-1,jmx-1]   = - lam[jmx-1];

    for j in range(1, jmx-1): 
        M[j,j-1] = lam[j]

        M[j,j]   = - (lam[j+1]+lam[j])

        M[j,j+1] = lam[j+1]


    Mdiv = M; #Divergence matrix 

    # Now solve climate change experiment
    #set up inital T profile 
    T = 0.5*(1-1*x*x);
    Tinit=T;
    Tglob = np.mean(Tinit);
    #T.shape
    Tglob_prev = -500
    # Timestepping loop
    for j in range(0, int(NMAX)-1):

        #print(np.mean(T)) 

        # spec. hum, and theta_e
        q = eps*relhum/psfc*e0*np.exp(a*(T+T_ctrl)/(b+(T+T_ctrl))); # here T is in oC. q is g kg-1

        theta_e = 1/cp*(cp*((T+T_ctrl)+273.15) + L*q); # note units of Kelvin are needed!!!

        theta_e_pert = theta_e-theta_e_ctrl; #anomalous theta_e

        # Calculate new T from Source and Sink terms.
        # Diffuse anomalous moist static energy (theta_e_pert)
        dT = delt/Cl*(R_frc - (B*T) + np.matmul(Mdiv,theta_e_pert));

        T = T + dT; 

        #print(np.mean(T))

        # Check to see if global mean energy budget has converged:   
        Fglob=np.mean(R_frc - (B*T))

        Tglob = np.mean(T)
        Tchange = Tglob - Tglob_prev
        if j % 50000 == 0:
            print(np.mean(T))
        if np.abs(Tchange) < 1.0e-30:
            break
        Tglob_prev = Tglob 

    divF_pert = -np.matmul(Mdiv,theta_e_pert);
    h_pert = theta_e_pert*cp;

    # Module to calculate terms in hyrological cycle

    q_expt = q; #sorry not sorry for all the ;s
    q_pert = q_expt-q_ctrl;
    #divF_pert = -Mdiv*theta_e_pert;
    h_pert = theta_e_pert*cp;
    T_pert = T;
    T_expt = T+T_ctrl;

    # define some things from the climatology first 

    Re = 6.4e6;                 # [m] earth's radius
    rho = 1e3;                  # [kg m-3] density

    wt = 1-norm.pdf(x, 0, np.sin(15*2*np.pi/360))/norm.pdf(0, 0, np.sin(15*2*np.pi/360)) # define Gaussian Hadley Cell width wt = 1-gaussmf(x,[sind(15) 0]);

    # calculate some things about the control climate (ctrl)

    Dhlf = Dmag*np.ones(jmx);                                   # [W m-2 K-1] calculate D on same grid as T,q, etc ##0.5*(D(1:end-1)+D(2:end));  #WARNING: would need to modify this if allowing D to vary with latitude
    h_ctrl = cp*(T_ctrl+273.15)+L*q_ctrl;                       # [J kg-1] control climate mse
    F_ctrl = -2*np.pi*np.square(Re)/cp*Dhlf*(1-np.square(x))*np.gradient(h_ctrl,x);  # [W] control climate flux 
    F_lh_ctrl = -2*np.pi*np.square(Re)/cp*Dhlf*(1-np.square(x))*np.gradient(L*q_ctrl,x);    # [W] control latent heat flux
    F_hc_ctrl = (1-wt)*F_ctrl;                                 # [W] Hadley Cell Flux
    heq_ctrl = h_ctrl[50];                                 #value at equator WARNING will have to change if change grid ; # [J kg-1] moist static energy at the surface
    V_ctrl = F_hc_ctrl/(heq_ctrl*1.06-h_ctrl);               # [kg s-1] Diagnosed mass transport in Hadley Cell (Nick's way)

    F_LH_ctrl = -L*V_ctrl*q_ctrl + wt*F_lh_ctrl;              # [W] latent heat (Hadley plus eddy)
    F_LH_eddy_ctrl = wt*F_lh_ctrl;                         # eddy latent heat fluxes including weighting function.
    divF_LH_ctrl = 1/(2*np.pi*np.square(Re))*np.gradient(F_LH_ctrl,x);         # [W m-2] divergence of latent heat flux
    E_m_P_ctrl = divF_LH_ctrl/(L*rho)*np.pi*1e7;                   # [m yr-1]E-P

    # code from Nick Siler to partition E-P;
    alpha=L/461.5/np.square(T_ctrl+273.15);                         # Nick's alpha parameter
    beta=cp/L/alpha/q_ctrl;                                   # beta parameter
    RG=180*(1*(1-np.square(x))-.4*np.exp(-np.square((x/.15))));                   # [W m-2] idealized R-G
    Ch=1.5e-3;                                                  # drag coefficient
    LWfb=0;                                                     # LW feedback at surface, in W/m2/K
    u=4+np.abs(np.sin(np.pi*x/1.5))*4; #-2.5*cos(3*np.arcsin(x));     # wind speed
    rho_air=1.2;       #psfc/287/(T_ctrl+273.15);               # air density
    E_ctrl=(RG*alpha+rho_air*cp*(1-relhum)*Ch*u)/(alpha+cp/L/q_ctrl);
    P_ctrl=E_ctrl-divF_LH_ctrl;
    # P_ctrl=P_ctrl/(L*rho)*np.pi*1e7; # [m yr-1]
    # E_ctrl=E_ctrl/(L*rho)*np.pi*1e7; # [m yr-1]

    # calculate some things about the new climate state (expt)

    h_expt = h_ctrl+h_pert;                                 # [J kg-1] new total mse
    F_expt = -2*np.pi*np.square(Re)/cp*Dhlf*(1-np.square(x))*np.gradient(h_expt,x);      # [W] new total flux
    F_lh_expt = -2*np.pi*np.square(Re)/cp*Dhlf*(1-np.square(x))*np.gradient(L*q_expt,x);    # [W] control latent heat flux
    F_hc_expt = (1-wt)*F_expt;                             # [W] Hadley Cell Flux
    heq_expt = h_expt[50];                                # [J kg-1] moist static energy at the surface
    V_expt = F_hc_expt/(heq_expt*1.06-h_expt);           # [wkg s-1] Diagnosed mass transport in Hadley Cell

    F_LH_expt = -L*V_expt*q_expt + wt*F_lh_expt;          # [W] latent heat (Hadley plus eddy)
    F_LH_eddy_expt = wt*F_lh_expt;                         # eddy latent heat fluxes including weighting function.
    divF_LH_expt = 1/(2*np.pi*np.square(Re))*np.gradient(F_LH_expt,x);      # [W m-2]
    E_m_P_expt = divF_LH_expt/(L*rho)*np.pi*1e7;               # [m yr-1] E-P

    alpha=L/461.5/np.square(T_expt+273.15);                         # Nick's alpha parameter
    beta=cp/L/alpha/q_expt;                                   # beta parameter
    #E_expt=(RG*alpha+rho_air*cp*(1-relhum)*Ch*u)/(alpha+cp/L/q_expt);
    #P_expt=E_expt-divF_LH_expt;
    #P_expt=P_expt/(L*rho)*np.pi*1e7; # [m yr-1]
    #E_expt=E_expt/(L*rho)*np.pi*1e7; # [m yr-1] 

    #ok, change implemented by GHR Mar
    #E_expt=(RG.*alpha+rho_air.*cp.*(1-relhum).*Ch.*u)./(alpha+cp./L./q_expt);
    # Equation 16 from Siler et al. (JClim,2018)
    Rv = 461; #gas constant for H20 vapor
    alf = L/(Rv*np.square(T_ctrl+273.15)); # Clausius-Clapeyron scaling 1/q*dq/dT
    beta = cp/(alf*q_ctrl*L); # Bowen ratio

    E_pert = (E_ctrl*T_pert*beta*(alf-2/(T_ctrl+273.15))-G_frc)/(1+beta);
    E_expt = E_ctrl+E_pert;

    P_expt=E_expt-divF_LH_expt;

    # convert from Wm-2 to m yr-1
    P_ctrl=P_ctrl/(L*rho)*np.pi*1e7; # [m yr-1]
    E_ctrl=E_ctrl/(L*rho)*np.pi*1e7; # [m yr-1]
    P_expt=P_expt/(L*rho)*np.pi*1e7; # [m yr-1]
    E_expt=E_expt/(L*rho)*np.pi*1e7; # [m yr-1] 
    
    divF_ctrl = 1/(2*np.pi*np.square(Re))*np.gradient(F_ctrl,x)
    divF_pert = 1/(2*np.pi*np.square(Re))*np.gradient(F_expt,x) - 1/(2*np.pi*np.square(Re))*np.gradient(F_ctrl,x)
    E_m_P_pert = E_m_P_expt - E_m_P_ctrl
    
    return T_ctrl, T_pert, divF_ctrl, divF_pert, E_m_P_ctrl, E_m_P_pert